<a href="https://colab.research.google.com/github/erlynnur/PRAKTIKUM-BIG-DATA-SEM-3/blob/main/Laprak_Big_Data_week_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
#Tugas 1
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('HandsOnPertemuan3').getOrCreate()

data = [('James', 'Sales', 3000),
        ('Michael', 'Sales', 4600),
        ('Robert', 'Sales', 4100),
        ('Maria', 'Finance', 3000),
        ('Abimanyu', 'Secretary', 6000)]
columns = ['EmployeeName', 'Department', 'Salary']
df = spark.createDataFrame(data, columns)
print("Dataframe yang telah dibuat :")
df.show()


Dataframe yang telah dibuat :
+------------+----------+------+
|EmployeeName|Department|Salary|
+------------+----------+------+
|       James|     Sales|  3000|
|     Michael|     Sales|  4600|
|      Robert|     Sales|  4100|
|       Maria|   Finance|  3000|
|    Abimanyu| Secretary|  6000|
+------------+----------+------+



In [14]:
#Tugas 2
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('HandsOnPertemuan3').getOrCreate()

data = [('James', 'Sales', 3000),
        ('Michael', 'Sales', 4600),
        ('Robert', 'Sales', 4100),
        ('Maria', 'Finance', 3000),
        ('Abimanyu', 'Secretary', 6000)]
columns = ['EmployeeName', 'Department', 'Salary']
print("hasil select employeename dan salary")
df.select('EmployeeName', 'Salary').show()
print("hasil filter salary > 3000")
df.filter(df['Salary'] > 3000).show()
print("hasil group by department dan avg salary")
df.groupBy('Department').avg('Salary').show()


hasil select employeename dan salary
+------------+------+
|EmployeeName|Salary|
+------------+------+
|       James|  3000|
|     Michael|  4600|
|      Robert|  4100|
|       Maria|  3000|
|    Abimanyu|  6000|
+------------+------+

hasil filter salary > 3000
+------------+----------+------+
|EmployeeName|Department|Salary|
+------------+----------+------+
|     Michael|     Sales|  4600|
|      Robert|     Sales|  4100|
|    Abimanyu| Secretary|  6000|
+------------+----------+------+

hasil group by department dan avg salary
+----------+-----------+
|Department|avg(Salary)|
+----------+-----------+
|     Sales|     3900.0|
|   Finance|     3000.0|
| Secretary|     6000.0|
+----------+-----------+



In [28]:
# Tugas 3: Eksplorasi tipe data kompleks
from pyspark.sql import SparkSession
from pyspark.sql.functions import struct, array

spark = SparkSession.builder.appName('HandsOnPertemuan3').getOrCreate()

data = [('James', 'Sales', 3000),
        ('Michael', 'Sales', 4600),
        ('Robert', 'Sales', 4100),
        ('Maria', 'Finance', 3000),
        ('Abimanyu', 'Secretary', 6000)]
columns = ['EmployeeName', 'Department', 'Salary']

df = spark.createDataFrame(data, columns)

# Buat kolom struct (gabungan beberapa kolom)
df = df.withColumn("Nama dan departemen", struct("EmployeeName", "Department"))

# Buat kolom array (gabungan nama dan gaji dalam list)
df = df.withColumn("nama dan gaji ", array("EmployeeName", "Salary"))

print("Hasil eksplorasi tipe data kompleks (Struct & Array)")
df.show(truncate=False)


Hasil eksplorasi tipe data kompleks (Struct & Array)
+------------+----------+------+---------------------+----------------+
|EmployeeName|Department|Salary|Nama dan departemen  |nama dan gaji   |
+------------+----------+------+---------------------+----------------+
|James       |Sales     |3000  |{James, Sales}       |[James, 3000]   |
|Michael     |Sales     |4600  |{Michael, Sales}     |[Michael, 4600] |
|Robert      |Sales     |4100  |{Robert, Sales}      |[Robert, 4100]  |
|Maria       |Finance   |3000  |{Maria, Finance}     |[Maria, 3000]   |
|Abimanyu    |Secretary |6000  |{Abimanyu, Secretary}|[Abimanyu, 6000]|
+------------+----------+------+---------------------+----------------+



In [25]:
# Tugas 4: Window Function - Ranking & Running Total
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql import functions as F

spark = SparkSession.builder.appName('HandsOnPertemuan3').getOrCreate()

data = [('James', 'Sales', 3000),
        ('Michael', 'Sales', 4600),
        ('Robert', 'Sales', 4100),
        ('Maria', 'Finance', 3000),
        ('Abimanyu', 'Secretary', 6000)]
columns = ['EmployeeName', 'Department', 'Salary']

df = spark.createDataFrame(data, columns)

# Window untuk ranking dalam tiap department
windowSpec = Window.partitionBy('Department').orderBy('Salary')

# Tambahkan ranking
df_rank = df.withColumn('Rank', F.rank().over(windowSpec))

print("Ranking berdasarkan Salary dalam tiap Department")
df_rank.show()

# Window untuk running total dalam tiap department
windowSpecRunning = Window.partitionBy('Department').orderBy('Salary').rowsBetween(Window.unboundedPreceding, Window.currentRow)

# Tambahkan running total
df_running = df.withColumn('RunningTotal', F.sum('Salary').over(windowSpecRunning))

print("Running Total Salary dalam tiap Department")
df_running.show()


Ranking berdasarkan Salary dalam tiap Department
+------------+----------+------+----+
|EmployeeName|Department|Salary|Rank|
+------------+----------+------+----+
|       Maria|   Finance|  3000|   1|
|       James|     Sales|  3000|   1|
|      Robert|     Sales|  4100|   2|
|     Michael|     Sales|  4600|   3|
|    Abimanyu| Secretary|  6000|   1|
+------------+----------+------+----+

Running Total Salary dalam tiap Department
+------------+----------+------+------------+
|EmployeeName|Department|Salary|RunningTotal|
+------------+----------+------+------------+
|       Maria|   Finance|  3000|        3000|
|       James|     Sales|  3000|        3000|
|      Robert|     Sales|  4100|        7100|
|     Michael|     Sales|  4600|       11700|
|    Abimanyu| Secretary|  6000|        6000|
+------------+----------+------+------------+



In [32]:
# Upload file ZIP manual
from google.colab import files
uploaded = files.upload()  # Pilih file precipitation-idn-csv-1.csv.zip

# Unzip file
import zipfile
import os

zip_path = list(uploaded.keys())[0]  # file zip yang di-upload
extract_path = "/content/precipitation_idn"
os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Lihat isi folder setelah unzip
print("Isi folder unzip:", os.listdir(extract_path))

# Pilih file CSV yang ada di dalam zip
csv_file = os.path.join(extract_path, os.listdir(extract_path)[0])
print("CSV file path:", csv_file)
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, max as Fmax, min as Fmin
import matplotlib.pyplot as plt
import pandas as pd
spark = SparkSession.builder.appName("AnalisisDatasetBesar").getOrCreate()

# Load CSV ke PySpark (ganti path sesuai dataset yang kamu unzip)
csv_file = "/content/precipitation_idn/precipitation-idn-csv-1.csv"
df = spark.read.csv(csv_file, header=True, inferSchema=True)

print("Schema dataset:")
df.printSchema()
print("Contoh data:")
df.show(5)

# Simpan data ke format parquet (lebih efisien untuk big data)
df.write.mode("overwrite").parquet("/content/output_parquet")

# Manipulasi Data
# Rata-rata curah hujan per tahun
from pyspark.sql.functions import year, to_date
df = df.withColumn("Date", to_date(col("date")))
df = df.withColumn("Year", year(col("Date")))

yearly = df.groupBy("Year").agg(avg("value").alias("AvgPrecipitation"))
yearly = yearly.orderBy("Year")

print("Rata-rata curah hujan per tahun:")
yearly.show()

# Tahun dengan curah hujan tertinggi & terendah
max_year = yearly.orderBy(col("AvgPrecipitation").desc()).limit(1)
min_year = yearly.orderBy(col("AvgPrecipitation").asc()).limit(1)

print("Tahun tertinggi:")
max_year.show()
print("Tahun terendah:")
min_year.show()

# Analisis rata-rata curah hujan per stasiun
station_avg = df.groupBy("station").agg(avg("value").alias("AvgPrecipitation"))
station_avg = station_avg.orderBy(col("AvgPrecipitation").desc())

print("10 Stasiun dengan curah hujan tertinggi:")
station_avg.show(10)

Saving precipitation-idn-csv-1.csv.zip to precipitation-idn-csv-1.csv (2).zip
Isi folder unzip: ['precipitation-idn-csv-1.csv']
CSV file path: /content/precipitation_idn/precipitation-idn-csv-1.csv
Schema dataset:
root
 |-- index: integer (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- datatype: string (nullable = true)
 |-- station: string (nullable = true)
 |-- value: integer (nullable = true)
 |-- fl_miss: integer (nullable = true)
 |-- fl_cmiss: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- indicator: string (nullable = true)

Contoh data:
+-----+-------------------+--------+-----------------+-----+-------+--------+---------+---------+
|index|               date|datatype|          station|value|fl_miss|fl_cmiss|  country|indicator|
+-----+-------------------+--------+-----------------+-----+-------+--------+---------+---------+
|    0|2010-01-01 00:00:00|    TPCP|GHCND:ID000096145|  109|      0|       0|Indonesia|     TPCP|
|    1|2010-01-01 00